# Выборы выборы, ...

## 1. Скачка данных с [сайта ЦИК](http://www.vybory.izbirkom.ru)

Изучаем структуру сайта. В ходе её изучения понимаем, что у каждого субъекта есть несколько больших изберательных комиссий, каждая из которых включает в себя какое-то количество УИКов.

Значит парсер должен иметь следущую структуру: 
1. Выбираем конкретные выборы
2. Переходим к конкретному субъекту
3. Переходим к конкретному участку
4. Собираем данные по УИКАМ

In [1]:
import requests                 # для отправки запросов
from bs4 import BeautifulSoup   # для поиска по html нужных элементов
import datetime                 # для работы с датами и временем
import time                     # для тайм-менеджмента

import numpy as np
import pandas as pd

from tqdm import tqdm_notebook

In [2]:
# Ссылка на конкретные выборы 
start_page_1 = "http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=100100031793509&vrn=100100031793505&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100031793509&type=227"

In [3]:
%%time

def subject_hrefer(url):
    '''
        Собирает все ссылки нс субъекты/районы 
        url : str
            ссылка для сбора ссылок 
    '''
    
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html,"lxml")
    
    # Поиск в html нужного куска
    pre_subjhrefs = soup.findAll("a",{'style':'text-decoration: none'})
    
    # Оставляем толкьо называния и ссылки
    subjects = [(item.text, item.get('href')) for item in pre_subjhrefs]

    # Возвращаем список объектов-ссылок.
    return list(set(subjects))  # Каждая ссылка собирается по два раза из-за дубляции заголовков у таблиц



# Собираем ссылки на субъекты 
subjects = subject_hrefer(start_page_1)
print('Собрано ссылок на субъекты:', len(subjects))

# Собираем по каждому субъекту ссылки на районы 
subj_subjects = { }
s = 0
for item in tqdm_notebook(subjects):
    cur_sub = subject_hrefer(item[1])
    s += len(cur_sub)
    subj_subjects[item[0]] = cur_sub
    
print('Собрано ссылок на районы:', s)

Собрано ссылок на субъекты: 85


HBox(children=(IntProgress(value=0, max=85), HTML(value='')))


Собрано ссылок на районы: 2744
CPU times: user 35.5 s, sys: 423 ms, total: 35.9 s
Wall time: 1min 36s


In [4]:
# Функция достаёт с каждой страницы хрефы на УИК
def region_href(url):
    '''
        Собирает все ссылки на УИКи
        url : str
            ссылка для сбора ссылок 
    '''
    
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html,"lxml")
    return soup.findAll('a')[-2].get('href')


def TableCreator(html_table):
    '''
        Делает из html разметки красивую табличку
        html_table : bs4
            html-разметка странички
    '''
    
    rows = html_table.find_all('tr')
    data = [ ]
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele])
    return(data)

def download_region(region):
    '''
        Скачивает информацию по конкретному региону 
        region: string
            имя региона (ключ в словаре из регионов)
    '''
    
    df_rr = [ ] # Вектор для таблиц 
    # Качаем инфомрацию по районам региона
    for item in subj_subjects[region]:
        urll = region_href(item[1])

        response = requests.get(urll)
        html = response.content
        soup = BeautifulSoup(html,"lxml")
        # Находим таблицы с УИК
        tablecontent = soup.find('table', {'style':"width:100%;border-color:#000000"})
        tablecontent = tablecontent.findAll('table')
        
        # Вынимаем таблицы
        Table1 = TableCreator(tablecontent[0])
        Table2 = TableCreator(tablecontent[1])
        
        df1 = pd.DataFrame(Table1[1:])
        df1.drop(0, axis=1, inplace=True)
        df1.columns = ['Имя колонки', 'Сумма']
        df1.drop('Сумма', axis=1, inplace=True)
        
        df2 = pd.DataFrame(Table2[1:])
        df2.columns = Table2[0]

        df_r = df1.join(df2)
        df_r = df_r.set_index('Имя колонки').T
        df_r['Район'] = df_r.shape[0]*[item[0]]  # добавляем колонку с названием района
        df_rr.append(df_r)                   # Собираем в вектор все таблицы по районам региона 
    
    # Создаём одну большую таблицу по региону
    big_df = pd.concat(df_rr)
    # Делаем колонку для конкретного региона
    big_df['Регион'] = big_df.shape[0] *[region]
    return big_df



In [5]:
dfs = [ ]
regions = list(subj_subjects.keys( ))

for reg in tqdm_notebook(regions):
    try:
        rg_df = download_region(reg)
        dfs.append(rg_df)
    except:
        print(reg)

HBox(children=(IntProgress(value=0, max=85), HTML(value='')))

Город Байконур (Республика Казахстан)
Территория за пределами РФ
Калининградская область



In [8]:
all_df = pd.concat(dfs)

In [9]:
all_df.shape
all_df.head( )

Имя колонки,"Число избирателей, включенных в список избирателей","Число избирательных бюллетеней, полученных участковой избирательной комиссией","Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно","Число избирательных бюллетеней, выданных в помещении для голосования в день голосования","Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования",Число погашенных избирательных бюллетеней,Число избирательных бюллетеней в переносных ящиках для голосования,Число бюллетеней в стационарных ящиках для голосования,Число недействительных избирательных бюллетеней,Число действительных избирательных бюллетеней,...,Число утраченных избирательных бюллетеней,"Число избирательных бюллетеней, не учтенных при получении",None,Жириновский Владимир Вольфович,Зюганов Геннадий Андреевич,Миронов Сергей Михайлович,Прохоров Михаил Дмитриевич,Путин Владимир Владимирович,Район,Регион
УИК №1039,405,330,0,263,7,60,7,263,3,267,...,0,0,None,19\n7.04%,58\n21.48%,10\n3.70%,54\n20.00%,126\n46.67%,Котельниковская городская,Московская область
УИК №1040,2684,2275,0,1666,26,583,26,1665,22,1669,...,0,0,None,149\n8.81%,295\n17.45%,70\n4.14%,250\n14.78%,905\n53.52%,Котельниковская городская,Московская область
УИК №1041,2375,2105,0,1408,48,649,48,1408,28,1428,...,0,0,None,70\n4.81%,248\n17.03%,60\n4.12%,188\n12.91%,862\n59.20%,Котельниковская городская,Московская область
УИК №1042,2049,1815,0,1156,104,555,104,1156,15,1245,...,0,0,None,124\n9.84%,211\n16.75%,47\n3.73%,108\n8.57%,755\n59.92%,Котельниковская городская,Московская область
УИК №1043,2102,1890,0,1334,47,509,47,1334,23,1358,...,0,0,None,102\n7.39%,209\n15.13%,56\n4.06%,146\n10.57%,845\n61.19%,Котельниковская городская,Московская область


In [10]:
all_df_1 = all_df[['Регион', 'Район', 'Число избирателей, включенных в список избирателей',
                   'Число избирательных бюллетеней, полученных участковой избирательной комиссией',
                  'Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно',
         'Число избирательных бюллетеней, выданных в помещении для голосования в день голосования',
       'Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования',
                                                       'Число погашенных избирательных бюллетеней',
                              'Число избирательных бюллетеней в переносных ящиках для голосования',
                                          'Число бюллетеней в стационарных ящиках для голосования',
                                                 'Число недействительных избирательных бюллетеней',
                                                   'Число действительных избирательных бюллетеней',
                                                  'Число полученных открепительных  удостоверений',
               'Число открепительных удостоверений, выданных избирателям на избирательном участке',
                             'Число избирателей, проголосовавших по открепительным удостоверениям',
                                             'Число неиспользованных открепительных удостоверений',
                                    'Число открепительных удостоверений, выданных избирателям ТИК',
                                                   'Число утраченных открепительных удостоверений',
                                                       'Число утраченных избирательных бюллетеней',
                                       'Число избирательных бюллетеней, не учтенных при получении'
                                                                                          ]]

In [11]:
names= ['Жириновский Владимир Вольфович', 'Зюганов Геннадий Андреевич', 'Миронов Сергей Михайлович',
        'Прохоров Михаил Дмитриевич', 'Путин Владимир Владимирович']

for nnm in names:
    all_df_1[nnm + ' (абсолютно)'] = all_df[nnm].apply(lambda x: x.split('\n')[0])
    all_df_1[nnm + ' (доля)'] = all_df[nnm].apply(lambda x: x.split('\n')[1])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
all_df_1

Имя колонки,Регион,Район,"Число избирателей, включенных в список избирателей","Число избирательных бюллетеней, полученных участковой избирательной комиссией","Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно","Число избирательных бюллетеней, выданных в помещении для голосования в день голосования","Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования",Число погашенных избирательных бюллетеней,Число избирательных бюллетеней в переносных ящиках для голосования,Число бюллетеней в стационарных ящиках для голосования,...,Жириновский Владимир Вольфович (абсолютно),Жириновский Владимир Вольфович (доля),Зюганов Геннадий Андреевич (абсолютно),Зюганов Геннадий Андреевич (доля),Миронов Сергей Михайлович (абсолютно),Миронов Сергей Михайлович (доля),Прохоров Михаил Дмитриевич (абсолютно),Прохоров Михаил Дмитриевич (доля),Путин Владимир Владимирович (абсолютно),Путин Владимир Владимирович (доля)
УИК №1039,Московская область,Котельниковская городская,405,330,0,263,7,60,7,263,...,19,7.04%,58,21.48%,10,3.70%,54,20.00%,126,46.67%
УИК №1040,Московская область,Котельниковская городская,2684,2275,0,1666,26,583,26,1665,...,149,8.81%,295,17.45%,70,4.14%,250,14.78%,905,53.52%
УИК №1041,Московская область,Котельниковская городская,2375,2105,0,1408,48,649,48,1408,...,70,4.81%,248,17.03%,60,4.12%,188,12.91%,862,59.20%
УИК №1042,Московская область,Котельниковская городская,2049,1815,0,1156,104,555,104,1156,...,124,9.84%,211,16.75%,47,3.73%,108,8.57%,755,59.92%
УИК №1043,Московская область,Котельниковская городская,2102,1890,0,1334,47,509,47,1334,...,102,7.39%,209,15.13%,56,4.06%,146,10.57%,845,61.19%
УИК №1044,Московская область,Котельниковская городская,2270,1940,0,1191,24,725,24,1191,...,107,8.81%,193,15.88%,38,3.13%,205,16.87%,649,53.42%
УИК №1045,Московская область,Котельниковская городская,3040,2545,0,1535,5,1005,5,1535,...,93,6.04%,225,14.61%,49,3.18%,326,21.17%,822,53.38%
УИК №1046,Московская область,Котельниковская городская,2572,2270,0,1471,36,763,36,1471,...,132,8.76%,299,19.84%,76,5.04%,200,13.27%,772,51.23%
УИК №1047,Московская область,Котельниковская городская,2664,2200,0,1855,47,298,47,1855,...,84,4.42%,325,17.09%,57,2.99%,133,6.99%,1262,66.35%
УИК №1048,Московская область,Красноармейская городская,2768,2600,0,1623,35,942,35,1622,...,116,7.00%,442,26.67%,83,5.01%,191,11.53%,786,47.44%


In [13]:
all_df_1.to_csv('pres_vote_2012.csv', sep='\t')

In [22]:
all_df_1[['Регион', 'Район', 'Число избирателей, включенных в список избирателей',
       'Число избирательных бюллетеней, полученных участковой избирательной комиссией',
       'Жириновский Владимир Вольфович (абсолютно)',
       'Зюганов Геннадий Андреевич (абсолютно)',
       'Миронов Сергей Михайлович (абсолютно)',
       'Прохоров Михаил Дмитриевич (абсолютно)',
       'Путин Владимир Владимирович (абсолютно)']].head(5)

Имя колонки,Регион,Район,"Число избирателей, включенных в список избирателей","Число избирательных бюллетеней, полученных участковой избирательной комиссией",Жириновский Владимир Вольфович (абсолютно),Зюганов Геннадий Андреевич (абсолютно),Миронов Сергей Михайлович (абсолютно),Прохоров Михаил Дмитриевич (абсолютно),Путин Владимир Владимирович (абсолютно)
УИК №1039,Московская область,Котельниковская городская,405,330,19,58,10,54,126
УИК №1040,Московская область,Котельниковская городская,2684,2275,149,295,70,250,905
УИК №1041,Московская область,Котельниковская городская,2375,2105,70,248,60,188,862
УИК №1042,Московская область,Котельниковская городская,2049,1815,124,211,47,108,755
УИК №1043,Московская область,Котельниковская городская,2102,1890,102,209,56,146,845


In [15]:
all_df_1.columns

Index(['Регион', 'Район', 'Число избирателей, включенных в список избирателей',
       'Число избирательных бюллетеней, полученных участковой избирательной комиссией',
       'Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно',
       'Число избирательных бюллетеней, выданных в помещении для голосования в день голосования',
       'Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования',
       'Число погашенных избирательных бюллетеней',
       'Число избирательных бюллетеней в переносных ящиках для голосования',
       'Число бюллетеней в стационарных ящиках для голосования',
       'Число недействительных избирательных бюллетеней',
       'Число действительных избирательных бюллетеней',
       'Число полученных открепительных  удостоверений',
       'Число открепительных удостоверений, выданных избирателям на избирательном участке',
       'Число избирателей, проголосовавших по открепительным удостоверениям',
       'Чи

Зависимость результатов выборов от явки. Ось абцисс - явка в процентах, ось ординат - процент голосов от общего зарегистрированного числа голосов на участке с таким показателем явки.

Объяснение 1: электорат оппозиции приходит весь, электорат проправительственной партии более пассивен. Поэтому их доля в явке может варьироваться в больших диапазонах.

Объяснение 2: вбросы, административное давление (рота солдат пришла голосовать, давление на врачей и других бюджетников) 

Гистограмма по России. Пики в красивых точках. 